In [1]:
#
# All code cells are hidden in the output by default
#

# Parameter cell. Wilk be replaced by export_backtest_report()
parameters = {}

In [2]:
#
# Setting up
#


# Loads strategy trades and universe as passed over
# by the host Python system as temp files
from tradeexecutor.backtest.tearsheet import BacktestReporter
reporter = BacktestReporter.setup_report(parameters)
state = reporter.get_state()
strategy_universe = universe = reporter.get_universe()


KeyboardInterrupt



In [ ]:
## Metric calculations

# Calculate different chart data and metrics.
from tradeexecutor.visual.equity_curve import calculate_equity_curve, calculate_returns

curve = calculate_equity_curve(state)
returns = calculate_returns(curve)
first_trade, last_trade = state.portfolio.get_first_and_last_executed_trade()
start_at = state.backtest_data.start_at
end_at = state.backtest_data.end_at
trades = list(state.portfolio.get_all_trades())
name = state.name

# Strategy backtest information

This notebook provides the information about the strategy performance

- The strategy backtesting methodology
- Benchmarking against cryptocurrency indexes
- Success of trading

In [ ]:
import pandas as pd

data = {
    "Strategy name": name,
    "Report created": state.created_at,
    "Backtesting period start": start_at,
    "Backtesting period end": end_at,
    "Trades": len(trades),
}

# Display dictionary as a pretty table output
# display(pd.DataFrame(data.items()).style.hide(axis="columns").hide(axis="index"))
display(pd.DataFrame(data.values(), index=data.keys()).style.hide(axis="columns"))

# Benchmark

This is the [benchmark](https://tradingstrategy.ai/glossary/benchmark) of this strategy.

Here we compare the returns and different risk metrics of the strategy performance:

- The strategy estimated performance based on this backtest
- Buy and hold -cryptocurrency indexes



In [ ]:
from tradeexecutor.analysis.multi_asset_benchmark import get_benchmark_data
from tradeexecutor.visual.benchmark import visualise_equity_curve_benchmark

benchmark_indexes = get_benchmark_data(
    strategy_universe,
    cumulative_with_initial_cash=state.portfolio.get_initial_cash()
)

fig = visualise_equity_curve_benchmark(
    name="This strategy",
    title="Strategy vs. indexes",
    portfolio_statistics=state.stats.portfolio,
    all_cash=state.portfolio.get_initial_cash(),
    benchmark_indexes=benchmark_indexes,
    height=800,
    log_y=False,
)

fig.show()

## Performance and risk metrics

Side-by-side comparison of strategy and buy and portfolio performance and risk metrics.

See [risk-adjusted return](https://tradingstrategy.ai/glossary/risk-adjusted-return) to learn more about how to compare risk and reward ratios of different trading strategies.


In [ ]:
from tradeexecutor.analysis.multi_asset_benchmark import compare_strategy_backtest_to_multiple_assets

compare_strategy_backtest_to_multiple_assets(
    state,
    strategy_universe,
    display=True,
)

# Equity curve

The equity curve allows to examine how stable the strategy profitability is.

Here we plot

- The strategy equity curve
- Maximum drawdown
- Daily profit

In [ ]:
from tradeexecutor.visual.equity_curve import visualise_equity_curve

visualise_equity_curve(returns)

# Monthly returns

Here we show the backtested returns by each month, and visualise the streaks of good and bad months.


In [ ]:
from tradeexecutor.visual.equity_curve import visualise_returns_over_time

visualise_returns_over_time(returns)

# Trading metrics

Overview of the performance of trades this strategy took.

- How many winning and losing trades we had
- How much on average each trade made

In [ ]:
from tradeexecutor.analysis.trade_analyser import build_trade_analysis

analysis = build_trade_analysis(state.portfolio)

summary_metrics = analysis.calculate_summary_statistics(state=state, time_bucket=universe.data_universe.time_bucket)
summary_metrics.display()

# More trading metrics

Trading metrics show how successfully the strategy trading is.

In [ ]:
if universe.has_lending_data():
    # Mixed spot, short, long
    trading_summary_html = analysis.calculate_all_summary_stats_by_side(state=state, time_bucket=universe.data_universe.time_bucket)
    with pd.option_context("display.max_row", None):
        display(trading_summary_html)
else:
    # Spot only
    analysis = build_trade_analysis(state.portfolio)
    trading_summary = analysis.calculate_summary_statistics()
    with pd.option_context("display.max_row", None):
        display(trading_summary.to_dataframe())


# Periodic return distribution

Show performance variations for different timeframes.

In [ ]:
from tradeexecutor.visual.equity_curve import visualise_returns_distribution

visualise_returns_distribution(returns)

# Individual trading position analysis

Examine the data of every individual trading position entry, exit and profitability.

In [ ]:
from tradeexecutor.analysis.trade_analyser import expand_timeline

timeline = analysis.create_timeline()

expanded_timeline, apply_styles = expand_timeline(
        universe.universe.exchanges,
        universe.universe.pairs,
        timeline)

# Do not truncate the row output
with pd.option_context("display.max_row", None):
    display(apply_styles(expanded_timeline))